# Data Aquisition

# **Music Data Processing and Analysis Pipeline** 🎵  

## **Overview**  
This Jupyter Notebook automates the process of collecting, processing, and analyzing music-related data. It integrates multiple data sources and techniques to gather track metadata, download audio files, extract audio features, and enrich the dataset with geographical information.  

## **Pipeline Workflow**  
The notebook is structured into the following key steps:  

1. **Data Collection** 📊  
   - Fetches metadata from Last.fm, MusicBrainz, and OpenCage API to gather track information, artist details, and geographic locations.  

2. **Audio Processing** 🎧  
   - Downloads audio tracks using `yt-dlp`.  
   - Extracts key audio features (MFCCs, Chroma, Tonnetz, Spectral Contrast, RMS, etc.) using `librosa`.  
   - Detects the dominant language of the track using OpenAI's `Whisper` model.  

3. **Data Integration & Storage** 🔄  
   - Combines track metadata with extracted audio features.  
   - Ensures missing geographic coordinates are retrieved and stored for future use.  
   - Saves the final dataset for further analysis or machine learning applications.

## **Expected Output** ✅  
- A CSV file (`all_data.csv`) containing enriched track metadata, audio features, and geographic information.  
- Processed and cleaned datasets ready for visualization, analysis, or machine learning applications.

## **Required Dependencies** ⚙️  
Ensure you have the following Python packages installed before running the notebook:  

In [ ]:
!pip install requests pandas python-dotenv opencage yt-dlp librosa pydub openai-whisper numpy

# Metadata Extraction for World Music Tracks

## Overview
This script collects metadata about world music tracks based on genre tags, artist information, and geographical data. It uses the Last.fm and MusicBrainz APIs to retrieve and filter data. The final output is saved as a CSV file for further analysis.

## Key Functionalities

### Dependencies and Setup
- **Required libraries**: `requests`, `pandas`, `time`, `os`, `json`, `dotenv`.
- **API keys and configuration**:
  - The script loads environment variables using `dotenv`.
  - An API key for Last.fm must be added.

### Loading Data
- Reads a `genre_mapping.json` file to map tags to specific genres.
- Loads `tag_to_country.json` to identify tags associated with countries or without country information.

### Functions
1. **`fetch_artist_location_from_musicbrainz`**:
   - Queries MusicBrainz to retrieve the location of an artist based on their MBID.
2. **`fetch_artist_info`**:
   - Retrieves artist details, including location and statistics, from Last.fm and MusicBrainz as a fallback.
3. **`fetch_top_tracks_by_tag`**:
   - Fetches top tracks for a specific genre tag from Last.fm.
4. **`should_add_track`**:
   - Ensures that the number of tracks per country does not exceed a specified limit.
5. **`get_country_from_tag`**:
   - Maps a genre tag to a corresponding country.
6. **`categorize_genre`**:
   - Categorizes tags into broader genres based on a predefined mapping.

### Data Processing
- Iterates over genre tags to fetch top tracks.
- Fetches artist information and attempts to map their location.
- Filters tracks to ensure a maximum limit per country.
- Assigns an incremental track ID and organizes track metadata into a structured format.

### Saving Output
- Aggregates all track metadata into a Pandas DataFrame.
- Saves the metadata as a CSV file (`world_music.csv`) inside a `data` folder.

## CSV File Structure
The generated CSV file contains the following columns:
- `track_id_number`: Incremental ID assigned to each track.
- `track_id`: Unique identifier from the Last.fm track URL.
- `track_name`: Name of the track.
- `artist_name`: Name of the artist.
- `location`: Location of the artist (country or area).
- `listeners`: Number of listeners for the track on Last.fm.
- `genre`: Generalized genre based on the tag.

## Notes

### Rate Limiting
To avoid exceeding API rate limits, the script includes delays between API calls (`time.sleep(0.5)`).

### File Structure
Ensure the following files are present in the same directory as the script:
- `genre_mapping.json`
- `tag_to_country.json`

### Customizing Limits
The `max_per_country` variable can be adjusted to set the maximum number of tracks per country.

## Output Example
The CSV file will include data similar to the following:

| track_id_number | track_id   | track_name       | artist_name | location    | listeners | genre     |
|------------------|------------|------------------|-------------|-------------|-----------|-----------|
| 1                | 12345      | Track A          | Artist X    | USA         | 1000      | Rock      |
| 2                | 67890      | Track B          | Artist Y    | UK          | 1500      | Pop       |



In [ ]:
import requests
import pandas as pd
import time
import os
import json
from dotenv import load_dotenv

load_dotenv()

API_KEY = '' # add API key here
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

# Load the genre mapping JSON
with open('genre_mapping.json') as f:
    genre_mapping = json.load(f)['genre_mapping']

# Load the JSON file for tags and countries
with open('tag_to_country.json') as f:
    tag_to_country = json.load(f)

# Get tags with countries and tags without countries
tags_with_country = tag_to_country.get('with_country', {})
tags_without_country = tag_to_country.get('without_country', [])
tags_country = tag_to_country.get('countries', {})

# Combine both lists of tags
tags = list(tags_with_country.keys()) + tags_without_country + list(tags_country.keys())

all_metadata = []
country_count = {}
max_per_country = 50

def fetch_artist_location_from_musicbrainz(mbid, retries=3, delay=3):
    """Fetch location from MusicBrainz using the artist's MBID."""
    url = f"https://musicbrainz.org/ws/2/artist/{mbid}?fmt=json"
    
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raises HTTPError for bad responses
            data = response.json()
            if data and isinstance(data, dict):
                area = data.get('area')
                if area and isinstance(area, dict):
                    location = area.get('name')
                    if location:
                        return location
                    else:
                        print(f"Warning: No location found for MBID {mbid}.")
            else:
                print(f"Warning: Invalid or empty data for MBID {mbid}.")
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == retries - 1:
                raise
    return None

def fetch_artist_info(artist_name, mbid=None):
    """Fetch artist info from Last.fm and fallback to MusicBrainz if needed."""
    params = {
        'method': 'artist.getInfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json'
    }
    url = "http://ws.audioscrobbler.com/2.0/"
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data and 'artist' in data:
            artist_info = data['artist']
            if 'mbid' in artist_info and not mbid:
                mbid = artist_info['mbid']
            if mbid:
                location = fetch_artist_location_from_musicbrainz(mbid)
                return location, artist_info
            else:
                print(f"Warning: No MBID found for artist {artist_name}.")
        else:
            print(f"Warning: Invalid or empty data for artist {artist_name}.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch artist info from Last.fm: {e}")
    except ValueError as e:
        print(f"JSON decoding failed: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return None, None

def fetch_top_tracks_by_tag(tag, limit=20):
    """Fetch top tracks by tag from Last.fm."""
    params = {
        'method': 'tag.getTopTracks',
        'tag': tag,
        'api_key': API_KEY,
        'format': 'json',
        'limit': limit
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.json().get('tracks', {}).get('track', [])
    else:
        print(f"Error fetching tracks for tag {tag}: {response.status_code}")
    return []

def should_add_track(location):
    """Determine if a track's location should be included based on max per country."""
    if location:
        country = location.split(',')[-1].strip() if ',' in location else location.strip()
        country_count[country] = country_count.get(country, 0)
        if country_count[country] < max_per_country:
            country_count[country] += 1
            return True
    return False

def get_country_from_tag(tag):
    """Return a country associated with a tag, if available."""
    return tags_country.get(tag) or tags_with_country.get(tag)

def categorize_genre(tag):
    """Categorize genre based on the tag."""
    tag_lower = tag.lower()
    
    for genre, tags in genre_mapping.items():
        if tag_lower in tags:
            return genre
    return 'Other'

# Initialize track ID counter
track_id_counter = 1

# Fetch tracks based on tags and filter by location
for tag in tags:
    tracks = fetch_top_tracks_by_tag(tag)
    for track in tracks:
        artist_name = track.get('artist', {}).get('name')
        mbid = track.get('artist', {}).get('mbid')  # Artist MBID if available
        
        # Fetch location and artist info (e.g., listeners)
        location, artist_info = fetch_artist_info(artist_name, mbid)
        
        # If no location, try mapping tag to country if the tag has an associated country
        if not location:
            location = get_country_from_tag(tag)
        
        # Get number of listeners
        listeners = artist_info.get('stats', {}).get('listeners', 0) if artist_info else 0
        
        # Categorize genre based on the tag
        genre = categorize_genre(tag)
        
        # Get the track URL to generate track ID
        track_url = track.get('url')
        track_id = track.get('url').split('/')[-1] if track_url else None
        
        # Add incremental track ID
        track_data = {
            'track_id_number': track_id_counter,  # Add the incremental ID
            'track_id': track_id,
            'track_name': track.get('name'),
            'artist_name': artist_name,
            'location': location,
            'listeners': listeners,
            'genre': genre
        }
        
        # Check if the track should be included
        if should_add_track(location):
            all_metadata.append(track_data)
            track_id_counter += 1  # Increment the track ID for the next track
        
        time.sleep(0.5)  # Avoid hitting the API too quickly

# Convert the metadata to a DataFrame
df = pd.DataFrame(all_metadata)

# Define the path to the data folder inside the current folder
data_folder = os.path.join(os.path.dirname(__file__), 'data')

# Ensure the data folder exists
os.makedirs(data_folder, exist_ok=True)

# Define the full path to the CSV file
csv_file_path = os.path.join(data_folder, 'world_music.csv')

# Save to a CSV file
df.to_csv(csv_file_path, index=False)
print(f"Metadata saved to {csv_file_path}")

# Adding Coordinates

This Python script adds geographic coordinates (latitude and longitude) based on the country or region of origin from a CSV file. It automates the process by using the **OpenCage Geocoder API** to fetch missing coordinates dynamically. The script merges data from two CSV files:

- **`diverse_world_music_with_listeners.csv`**: Contains music tracks with a `location` column representing the country of origin.
- **`country_coordinates.csv`**: Contains country and region names along with their corresponding geographic coordinates (latitude and longitude).

The script performs the following tasks:

1. **Load CSV Files**: It loads the `diverse_world_music_with_listeners.csv` and `country_coordinates.csv` files into pandas DataFrames.
2. **Clean Data**: Ensures that the `Country` column in the `country_coordinates.csv` is free from extra spaces.
3. **Merge Data**: Merges the two DataFrames based on the country names (`location` from `diverse_world_music_with_listeners.csv` and `Country` from `country_coordinates.csv`).
4. **Handle Missing Coordinates**: If coordinates for a country are missing in the `country_coordinates.csv`, the script:
   - Fetches the missing latitude and longitude automatically using the **OpenCage Geocoder API**.
   - Adds the new coordinates to both the DataFrame and the `country_coordinates.csv` file for future reference.
5. **Save Updated Data**: The script saves:
   - The updated coordinates to the `country_coordinates.csv` file.
   - The merged DataFrame (with coordinates) to a new CSV file, `tracks_with_coordinates.csv`.

---

### Workflow

1. **Loading Files**:
   - The script loads `world_music_data.csv` and `country_coordinates.csv` using `pandas.read_csv()`.

2. **Cleaning the Data**:
   - It strips extra spaces from the `Country` column in the `country_coordinates.csv` file to ensure accurate matching with the `location` column in `diverse_world_music_with_listeners.csv`.

3. **Merging the Data**:
   - The `merge()` function is used to combine the two DataFrames based on the country names.

4. **Handling Missing Coordinates**:
   - If any rows have missing latitude and longitude, the script uses the OpenCage Geocoder API to automatically fetch the missing coordinates.
   - The fetched coordinates are appended to the `country_coordinates.csv` file for future use, ensuring the database remains up-to-date.

5. **Saving the Updated Data**:
   - The merged DataFrame, which now contains the coordinates, is saved as `tracks_with_coordinates.csv`.
   - The updated `country_coordinates.csv` file is saved with the newly fetched coordinates.

---

### Key Features

- **Automated Retrieval**: Missing coordinates are fetched dynamically using the OpenCage Geocoder API, removing the need for manual input.
- **Reusable Reference File**: Newly fetched coordinates are appended to `country_coordinates.csv`, ensuring they are available for subsequent runs of the script.
- **Scalable Solution**: Works efficiently for large datasets, automatically resolving missing geographic data.

---

### Example Workflow

1. Run the script, and if any missing coordinates are found, it will use the OpenCage Geocoder API to fetch them automatically.
2. Updated coordinates will be saved to the reference file (`country_coordinates.csv`) for future use.

### Requirements

- **API Key**: Obtain an OpenCage Geocoder API key from [OpenCage Geocoder](https://opencagedata.com/).
- **Install Required Libraries**: Use `pip install opencage pandas`.

### Example Output

When the script encounters a missing coordinate, it fetches it automatically and updates the files:

```plaintext
Fetching coordinates for missing country or region: France
Coordinates for France added and saved: (48.8566, 2.3522)


In [12]:
import os
import pandas as pd
from opencage.geocoder import OpenCageGeocode

# Your OpenCage API key
API_KEY = '9aba218e5bca4aad9afdcc9337292597'  # Replace with your OpenCage API key
geocoder = OpenCageGeocode(API_KEY)

# Use the current working directory as the base path
current_dir = os.getcwd()
data_folder = os.path.join(current_dir, 'data')
data_references_folder = os.path.join(current_dir, 'data_references')

# Load the CSV files
tracks_df = pd.read_csv(os.path.join(data_folder, 'world_music.csv'))
coords_csv_path = os.path.join(data_references_folder, 'country_coordinates.csv')
coords_df = pd.read_csv(coords_csv_path)

# Ensure both DataFrames have a clean 'Country' column
coords_df['Country'] = coords_df['Country'].str.strip()  # Remove extra spaces

# Function to update the merged DataFrame
def update_merged_df():
    return pd.merge(tracks_df, coords_df, left_on='location', right_on='Country', how='left')

# Function to fetch coordinates using OpenCage
def fetch_coordinates(country_name):
    try:
        results = geocoder.geocode(country_name)
        if results:
            latitude = results[0]['geometry']['lat']
            longitude = results[0]['geometry']['lng']
            return latitude, longitude
        else:
            print(f"No coordinates found for {country_name}.")
            return None, None
    except Exception as e:
        print(f"Error fetching coordinates for {country_name}: {e}")
        return None, None

# Initial merge
merged_df = update_merged_df()

# Check for missing coordinates and fetch them automatically
while merged_df['Latitude'].isna().any():
    for index, row in merged_df[merged_df['Latitude'].isna()].iterrows():
        country = row['location']
        print(f"Fetching coordinates for missing country or region: {country}")
        
        # Fetch coordinates from OpenCage
        latitude, longitude = fetch_coordinates(country)
        if latitude is not None and longitude is not None:
            # Add the new country and coordinates to the coords_df DataFrame
            new_row = pd.DataFrame([{'Country': country, 'Latitude': latitude, 'Longitude': longitude}])
            coords_df = pd.concat([coords_df, new_row], ignore_index=True)
            
            # Save the updated coordinates back to the CSV file
            coords_df.to_csv(coords_csv_path, index=False)
            print(f"Coordinates for {country} added and saved: ({latitude}, {longitude})")
            
            # Break the loop to re-merge with the updated coordinates
            break
        else:
            print(f"Failed to fetch coordinates for {country}. Please check the name or API key.")
    
    # Re-merge the DataFrames after updating the coordinates
    merged_df = update_merged_df()

# Save the merged DataFrame with coordinates to a new CSV file in the data folder
output_csv_path = os.path.join(data_folder, 'tracks_with_coordinates.csv')
merged_df.to_csv(output_csv_path, index=False)

print(f"Metadata with coordinates saved to {output_csv_path}")


Fetching coordinates for missing country or region: Boulder
Coordinates for Boulder added and saved: (40.0149856, -105.270545)
Fetching coordinates for missing country or region: Toronto
Coordinates for Toronto added and saved: (43.6534817, -79.3839347)
Fetching coordinates for missing country or region: Soviet Union
Coordinates for Soviet Union added and saved: (48.2582473, 14.5007542)
Fetching coordinates for missing country or region: St. Louis
Coordinates for St. Louis added and saved: (38.6280278, -90.1910154)
Fetching coordinates for missing country or region: Türkiye
Coordinates for Türkiye added and saved: (38.9597594, 34.9249653)
Fetching coordinates for missing country or region: Lund
Coordinates for Lund added and saved: (55.7029296, 13.1929449)
Fetching coordinates for missing country or region: Northern Ireland
Coordinates for Northern Ireland added and saved: (54.5859836, -6.9591554)
Fetching coordinates for missing country or region: Ibiza
Coordinates for Ibiza added and

# Music Track Feature Extraction and Language Detection

This notebook demonstrates how to download music tracks, extract various audio features, and detect the language of the song. The extracted features and detected language are then saved to a CSV file for further analysis.

## Overview

The code achieves the following:

1. **Download Music Tracks**: It searches for and downloads music tracks from YouTube based on track names and artist names using `yt-dlp`.
2. **Extract Audio Features**: It processes the downloaded tracks to extract a set of audio features such as MFCCs (Mel-frequency cepstral coefficients), chroma features, spectral contrast, tonnetz, RMS (Root Mean Square), spectral rolloff, zero-crossing rate, and the track's duration.
3. **Detect Language**: It uses the `whisper` model to transcribe the audio and detect the language of the song by processing a trimmed segment of the audio file.
4. **Store Results**: The extracted features, along with the detected language, are stored in a CSV file for future analysis.
5. **Clean-up**: The processed audio files are deleted to save storage.

## Steps in the Process

### 1. **Paths and Setup**

The code sets up necessary directories and paths:
- `csv_file`: Path to the CSV file containing track information (track names, artist names, etc.).
- `output_dir`: Directory where the downloaded songs will be stored.
- `data_folder`: Directory for storing feature data.
- `output_csv`: Path for saving the extracted audio features and detected language in CSV format.

### 2. **Loading the Dataset**

The dataset (`world_music.csv`) containing metadata for each track is loaded into a DataFrame. The script checks that the necessary columns (`track_id_number`, `track_id`, `track_name`, `artist_name`) are present in the dataset. If any of these columns are missing, an error is raised.

### 3. **Downloading Songs**

The function `download_song` is responsible for downloading each song using `yt-dlp`. It constructs a search query from the track name and artist name and downloads the best audio quality available. If the song has already been downloaded, the function skips the download and moves to the next track.

### 4. **Extracting Audio Features**

Once the song is downloaded, the function `extract_features` extracts a variety of audio features from the song using the `librosa` library. These features include:
- **MFCCs**: Mel-frequency cepstral coefficients, commonly used for audio processing and speech recognition.
- **Chroma**: Represents the harmonic content of the audio.
- **Spectral Contrast**: Measures the difference in amplitude between peaks and valleys in a spectrum.
- **Tonnetz**: A set of features used to capture harmonic relations in music.
- **RMS**: Root mean square, a measure of the signal’s energy.
- **Spectral Rolloff**: The frequency below which a certain percentage of the total spectral energy lies.
- **Zero-Crossing Rate**: The rate at which the signal changes sign.
- **Length**: The duration of the audio track.

### 5. **Detecting the Language of the Song**

The function `detect_language` detects the language of the song by transcribing a trimmed segment (the first 30 seconds) of the song using the `whisper` model. It then returns the detected language. If the transcription is not clear, it returns "No Speech" or "Unknown" as appropriate.

### 6. **Saving the Extracted Features and Language**

The extracted features, along with the track ID and detected language, are saved to a CSV file (`audio_features.csv`). Each row in the CSV file represents one track, and the columns contain the track ID and the extracted features.

### 7. **Clean-up**

After processing each song, the downloaded audio file is deleted to conserve storage. The program moves on to the next track in the dataset.

### 8. **Completion**

Once all the tracks in the dataset have been processed, a message is printed indicating that all tracks have been successfully processed.

## Conclusion

This code provides an efficient way to download music tracks, extract relevant audio features, detect the language of the songs, and store this information for further analysis. This can be useful for tasks such as building a machine learning model for music classification or genre prediction, or for understanding how different audio features correlate with the language of the song.

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import csv
import librosa
from yt_dlp import YoutubeDL
from pydub import AudioSegment
import whisper

# Paths and setup
csv_file = 'data/world_music.csv'
output_dir = 'downloaded_songs'
data_folder = os.path.join(os.getcwd(), 'data')
output_csv = os.path.join(data_folder, 'audio_features.csv')

os.makedirs(output_dir, exist_ok=True)
os.makedirs(data_folder, exist_ok=True)

# Load dataset
df = pd.read_csv(csv_file)
required_columns = ['track_id_number', 'track_id', 'track_name', 'artist_name']
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

# Load Whisper model
model = whisper.load_model("small")

def download_song(track_id_number, track_name, artist_name):
    """Download song using yt-dlp."""
    search_query = f"{track_name} {artist_name}"
    output_path = os.path.join(output_dir, f'{track_id_number}.wav')
    if os.path.exists(output_path):  # Skip if already downloaded
        print(f"Song {track_name} by {artist_name} already exists. Skipping download.")
        return output_path

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_dir, f'{track_id_number}.%(ext)s'),
        'quiet': False,
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            },
        ],
    }

    with YoutubeDL(ydl_opts) as ydl:
        try:
            print(f"Downloading: {search_query}")
            ydl.download([f"ytsearch1:{search_query}"])
            print(f"Downloaded {track_name} by {artist_name}.")
            return output_path
        except Exception as e:
            print(f"Failed to download {track_name} by {artist_name}: {e}")
            return None

def extract_features(audio_file):
    """Extract audio features from a file."""
    try:
        y, sr = librosa.load(audio_file)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr), axis=1)
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1)
        tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr), axis=1)
        rms = np.mean(librosa.feature.rms(y=y))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y))
        length = librosa.get_duration(y=y, sr=sr)
        return np.concatenate([mfccs, chroma, spectral_contrast, tonnetz, [rms], [rolloff], [zcr], [length]])
    except Exception as e:
        print(f"Error extracting features from {audio_file}: {e}")
        return None

def detect_language(audio_file):
    """Detect language of an audio file."""
    try:
        audio = AudioSegment.from_wav(audio_file)
        trimmed_audio = audio[30 * 1000:]  # Trim first 30 seconds
        temp_file = "temp_trimmed.wav"
        trimmed_audio.export(temp_file, format="wav")
        result = model.transcribe(temp_file, verbose=False, language=None, task="transcribe")
        os.remove(temp_file)
        transcription = result.get('text', '').strip()
        if not transcription:
            return "No Speech"
        return result.get('language', 'Unknown')
    except Exception as e:
        print(f"Error detecting language for {audio_file}: {e}")
        return None

# Prepare the CSV for writing features
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['File Number'] + \
             [f'MFCC{i+1}' for i in range(20)] + \
             [f'Chroma{i+1}' for i in range(12)] + \
             [f'SpectralContrast{i+1}' for i in range(7)] + \
             [f'Tonnetz{i+1}' for i in range(6)] + \
             ['RMS', 'SpectralRolloff', 'ZeroCrossingRate', 'Length', 'Language']
    writer.writerow(header)

# Process each song
for _, row in df.iterrows():
    track_id_number = row['track_id_number']
    track_name = row['track_name']
    artist_name = row['artist_name']

    # Download song
    audio_file = download_song(track_id_number, track_name, artist_name)
    if not audio_file:
        continue

    # Extract features
    features = extract_features(audio_file)
    if features is None:
        continue

    # Detect language
    language = detect_language(audio_file)

    # Save features and language to CSV
    with open(output_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        row_data = [track_id_number] + features.tolist() + [language]
        writer.writerow(row_data)

    # Remove processed file to save storage
    os.remove(audio_file)
    print(f"Processed and deleted: {audio_file}")

print("All tracks processed successfully.")


/opt/conda/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Song Saoko by Rosalía already exists. Skipping download.
Detected language: Spanish


100%|██████████| 11020/11020 [00:09<00:00, 1102.43frames/s]


Processed and deleted: downloaded_songs/1.wav
Song LA FAMA by Rosalía already exists. Skipping download.
Detected language: Spanish


100%|██████████| 22137/22137 [00:03<00:00, 5819.57frames/s]


...

[youtube:search] query "Fiura Lugua Centeno" page 1: Downloading API JSON
[youtube:search] Playlist Fiura Lugua Centeno: Downloading 1 items of 1
[download] Downloading item 1 of 1
[youtube] Extracting URL: https://www.youtube.com/watch?v=DLIOsLxkhlI
[youtube] DLIOsLxkhlI: Downloading webpage
[youtube] DLIOsLxkhlI: Downloading tv client config
[youtube] DLIOsLxkhlI: Downloading player 1080ef44
[youtube] DLIOsLxkhlI: Downloading tv player API JSON
[youtube] DLIOsLxkhlI: Downloading ios player API JSON
[youtube] DLIOsLxkhlI: Downloading m3u8 information
[info] DLIOsLxkhlI: Downloading 1 format(s): 251
[download] Destination: downloaded_songs/1744.webm
[download] 100% of    3.45MiB in 00:00:00 at 4.52MiB/s   
[ExtractAudio] Destination: downloaded_songs/1744.wav
Deleting original file downloaded_songs/1744.webm (pass -k to keep)
[download] Finished downloading playlist: Fiura Lugua Centeno
Downloaded Fiura by Lugua Centeno.
Detected language: Yoruba


 84%|████████▎ | 15205/18205 [00:26<00:05, 578.12frames/s] 

Processed and deleted: downloaded_songs/1744.wav
Downloading: Punta Cartel - Bounile Punta Cartel
[youtube:search] Extracting URL: ytsearch1:Punta Cartel - Bounile Punta Cartel
[download] Downloading playlist: Punta Cartel - Bounile Punta Cartel
[youtube:search] query "Punta Cartel - Bounile Punta Cartel": Downloading web client config


[youtube:search] query "Punta Cartel - Bounile Punta Cartel" page 1: Downloading API JSON
[youtube:search] Playlist Punta Cartel - Bounile Punta Cartel: Downloading 1 items of 1
[download] Downloading item 1 of 1
[youtube] Extracting URL: https://www.youtube.com/watch?v=j25MB8P5DJQ
[youtube] j25MB8P5DJQ: Downloading webpage
[youtube] j25MB8P5DJQ: Downloading tv client config
[youtube] j25MB8P5DJQ: Downloading player 1080ef44
[youtube] j25MB8P5DJQ: Downloading tv player API JSON
[youtube] j25MB8P5DJQ: Downloading ios player API JSON
[youtube] j25MB8P5DJQ: Downloading m3u8 information
[info] j25MB8P5DJQ: Downloading 1 format(s): 251
[download] Destination: downloaded_songs/1745.webm
[download] 100% of    6.89MiB in 00:00:00 at 8.15MiB/s     
[ExtractAudio] Destination: downloaded_songs/1745.wav
Deleting original file downloaded_songs/1745.webm (pass -k to keep)
[download] Finished downloading playlist: Punta Cartel - Bounile Punta Cartel
Downloaded Punta Cartel - Bounile by Punta Cartel.

 15%|█▍        | 7486/51269 [02:23<11:41, 62.41frames/s] 

# Combining Track Metadata and Audio Features

This section of the code demonstrates how to combine two CSV files containing music track metadata and audio features. After merging the two datasets on a common identifier, the result is saved to a new CSV file for further analysis.

## Overview

The code performs the following steps:

1. **Load Track Metadata and Audio Features**: It loads two CSV files:
   - `tracks_with_coordinates.csv`: Contains metadata about tracks, including track IDs.
   - `audio_features.csv`: Contains extracted audio features of the tracks.
2. **Ensure Column Consistency**: It ensures that the columns used for merging (track ID) are of the same data type (integer).
3. **Merge the DataFrames**: It merges the two datasets on the `track_id` and `File Number` columns.
4. **Remove Redundant Columns**: It removes the redundant column (`File Number`) from the merged DataFrame.
5. **Save the Combined Data**: The combined DataFrame is saved to a new CSV file (`all_data.csv`) in the data folder.

## Code Walkthrough

### 1. **Load the CSV Files**

The `tracks_with_coordinates.csv` and `audio_features.csv` files are loaded into Pandas DataFrames:

```python
tracks_df = pd.read_csv(os.path.join(data_folder, 'tracks_with_coordinates.csv'))
audio_features_df = pd.read_csv(os.path.join(data_folder, 'audio_features.csv'))


In [ ]:
import pandas as pd
import os

# Define the path to the data folder (relative to the current working directory)
data_folder = os.path.join(os.getcwd(), 'data')

# Load the CSV files from the data folder
tracks_df = pd.read_csv(os.path.join(data_folder, 'tracks_with_coordinates.csv'))
audio_features_df = pd.read_csv(os.path.join(data_folder, 'audio_features.csv'))

# Ensure the columns are of the same type for merging
tracks_df['track_id'] = tracks_df['track_id'].astype(int)
audio_features_df['File Number'] = audio_features_df['File Number'].astype(int)

# Merge the DataFrames on 'track_id' and 'File Number'
combined_df = pd.merge(tracks_df, audio_features_df, left_on='track_id', right_on='File Number', how='inner')

# Drop the redundant 'File Number' column
combined_df = combined_df.drop(columns=['File Number'])

# Save the combined DataFrame to a new CSV file in the data folder
combined_csv_path = os.path.join(data_folder, 'all_data.csv')
combined_df.to_csv(combined_csv_path, index=False)

print(f"Combined data saved to {combined_csv_path}")